# <b>Все ли мы такие разные</b>
### _или как найти похожего на себя пользователя_

## Задача

Написать функцию, которая решает задачу <b>поиска пользователя, наиболее похожего на нового пользователя</b> по их характеристикам. Смысловая задача определить, какой из существующих пользователей <b>users_stats</b> имеет наиболее схожее поведение/предпочтения с новым пользователем <b>new_user_stats</b>.

### Зачем это нужно
- <b>Рекомендательные системы</b> и поиск похожих пользователей для рекомендаций (например: если новый пользователь похож на пользователя X, ему можно предлагать тот же контент)
- <b>Кластеризация данных</b> и отнесение нового пользователя к группе
- Анализ пользовательских профилей и <b>поиск аномалий</b> если сходство с всеми пользователями низкое, это может быть подозрительный аккаунт


## Данные

Имеется таблица покупок в интернет-магазине из 10 пользователей, где первый столбец это <b>User ID</b> пользователя, а остальные столбцы — активности по категориям или количество покупок категорий товаров этим пользователем.

| User ID | Смартфон | Телевизор | Ноутбук | Шкаф | Холодильник | Посуда |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 1 | 2 | 1 | 0 | 0 | 0 | 0 |
| 2 | 1 | 1 | 2 | 1 | 0 | 0 |
| 3 | 2 | 0 | 1 | 0 | 0 | 0 |
| 4 | 1 | 1 | 2 | 1 | 0 | 1 |
| 5 | 0 | 0 | 1 | 2 | 0 | 0 |
| 6 | 0 | 0 | 0 | 0 | 0 | 5 |
| 7 | 1 | 0 | 0 | 0 | 0 | 0 |
| 8 | 0 | 1 | 1 | 0 | 0 | 0 |
| 9 | 0 | 0 | 0 | 1 | 1 | 3 |
| 10 | 1 | 0 | 0 | 2 | 1 | 4 |

Далее представим эту таблицу в виде <b>numpy array</b> матрицы <b>users_stats:</b> из 10 пользователей, где каждый описан 6 числами активности по категориям.

In [85]:
import numpy as np

users_stats = np.array(
    [
        [2, 1, 0, 0, 0, 0],
        [1, 1, 2, 1, 0, 0],
        [2, 0, 1, 0, 0, 0],
        [1, 1, 2, 1, 0, 1],
        [0, 0, 1, 2, 0, 0],
        [0, 0, 0, 0, 0, 5],
        [1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 3],
        [1, 0, 0, 2, 1, 4]
    ], 
)

Новый посетитель интернет-магазина, о покупках которого известно следующее

| Смартфон | Телевизор | Ноутбук | Шкаф | Холодильник | Посуда |
| :---: | :---: | :---: | :---: | :---: | :---: |
| 0 | 1 | 2 | 0 | 0 | 0 |

Данные о новом пользователе тоже представляем в виде <b>numpy array</b> матрицы <b>new_user_stats:</b> с 6 характеристиками.

In [86]:
new_user_stats = np.array([0, 1, 2, 0, 0, 0])

## Обоснование
### Каждый человек это вектор
Представим, что мы пытаемся понять, насколько два человека похожи друг на друга, основываясь на их вкусах. Чтобы реализовать алгоритм сравнения пользователей нам понадобиться линейная алгебра (скалярное произведение, нормы) и векторное представление наших данных о пользователях.
\begin{equation*}
\LARGE
\vec{a} = [a_1,a_2,\dots,a_n]
\end{equation*}
<p style="text-align:center;">или</p>
\begin{equation*}
\LARGE
\vec{next\textunderscore user\textunderscore stats} = [0,~1,~2,~0,~0,~0]
\end{equation*}
Каждый человек — это стрелка, он же вектор, в многомерном пространстве, где каждая ось — категория активности (смартфон, телевизор, ноутбук и т.д.). По сути матрица <b>users_stats</b> это 10 векторов в 6-мерном пространстве, а <b>new_user_stats</b> это всего лишь один вектор в таком же 6-мерном пространстве.
\begin{equation*}
\LARGE
users\textunderscore stats_{m,n} = 
\begin{matrix}
  \vec{a_{1}} \\
  \vec{a_{2}} \\
  \vdots \\
  \vec{a_{m}} 
 \end{matrix} =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n} 
 \end{pmatrix}
\end{equation*}
     
где:<br>
$m$ — количество пользователей в данных интернет-магазина,<br>
$n$ — количество столбцов по количеству категорий активности.

### Мера схожести

Вернемся к нашему рассуждению, что каждый человек это стрелка и зададим вопрос:

#### 1. Как же измерить схожесть направления стрелок?
- Если стрелки указывают в одном направлении — <b>вкусы пропорционально одинаковы</b>
- Если стрелки перпендикулярны — вкусы <b>не связаны</b>
- Если стрелки направлены противоположно — вкусы <b>противоположны</b>

#### 2. Угол между стрелками
- Угол между двумя стрелками — это <b>мера их схожести</b>
- Чем меньше угол, тем ближе стрелки направлены — тем более похожи вкусы

#### 3. Как вычислить угол?
- Из школьного курса геометрии мы знаем, что косинус угла между двумя векторами можно найти через их <b>скалярное произведение</b> и длины по формуле косинусного сходства:
\begin{equation*}
\LARGE
cos(\theta) = \frac{\vec{a} \space \vec{b}}{|\vec{a}| \space |\vec{b}|}
\end{equation*}
где:<br> $\vec{a}$ и $\vec{b}$ — векторы нового пользователя <b>new_user_stats</b> и существующего пользователя из базы данных.
<br>
</br>
- Для наглядности дальнейших вычислений нашу формулу косинусного сходства можно выразить ещё вот так:
\begin{equation*}
\LARGE
cos(\theta) = \frac{скалярное~произведение}{|норма~нового~пользователя| \space\times\space |норма~текущего~пользователя|}
\end{equation*}

### Скалярное произведение

Скалярное произведение для двух векторов в $N$-мерном пространстве выглядит так:

\begin{equation*}
\LARGE
\vec{a} \space \vec{b} = a_1 \space b_1 + a_2 \space b_2 + \dots + a_n \space b_n = \displaystyle\sum_{i=1}^{n} a_i \space b_i
\end{equation*}
где:<br>
$ \vec{a} = [a_1,a_2,\dots,a_n]$ — вектор в $N$-мерном пространстве,<br>
$ \vec{b} = [b_1,b_2,\dots,b_n]$ — вектор в $N$-мерном пространстве.

### Евклидова норма
Норма вектора, его длина, вычисляется по формуле евклидовой нормы, которая является обобщением теоремы Пифагора для многомерного пространства.

- В двумерном пространстве длина вектора $[x, y]$ вычисляется как $\sqrt{x^2 + y^2}$ (по теореме Пифагора)
- В трёхмерном пространстве для вектора $[x, y, z]$ длина будет $\sqrt{x^2 + y^2 + z^2}$
- Для вектора с $N$ измерениями:
\begin{equation*}
\LARGE
|\vec{a}| = \sqrt{a^2_1+a^2_1+\dots+a^2_n}
\end{equation*}
где:<br>
$ \vec{a} = [a_1,a_2,\dots,a_n]$ — вектор в $N$-мерном пространстве,<br>
$ |\vec{a}|$ — его длина (евклидова норма).


## Примеры расчетов

#### Случай 1: Векторы сонаправлены
- Новый пользователь <b>new_user_stats [0, 1, 2, 0, 0, 0]</b>.
- Пользователь из базы <b>[0, 3, 6, 0, 0, 0]</b> (в 3 раза активнее, но пропорции те же).
- Вычисляем их скалярное произведение:
\begin{equation*}
0 × 0 + 1 × 3 + 2 × 6 + 0 × 0 + 0 × 0 + 0 × 0 = 3 + 12 = 15.
\end{equation*}
- Косинусное сходство:
\begin{equation*}
cos(\theta) = \frac{15}{\sqrt{5}\space × \sqrt{45}} = \frac{15}{15} = 1 \space (идеальное \space сходство)
\end{equation*}

#### Случай 2: Векторы перпендикулярны
- Новый пользователь <b>new_user_stats [0, 1, 2, 0, 0, 0]</b>.
- Пользователь из базы: <b>[0, 0, 0, 5, 0, 0]</b> (активен в другой категории).
- Вычисляем их скалярное произведение:
\begin{equation*}
0 × 0 + 1 × 0 + 2 × 0 + 0 × 5 + 0 × 0 + 0 × 0 = 0
\end{equation*}
- Косинусное сходство:
\begin{equation*}
cos(\theta) = 0 \space (нет \space связи)
\end{equation*}

## Код на языке Python

- Для вычисления скалярного произведения векторов над понадобится метод dot() из библиотеки NumPy.
- Евклидову норму, длину векторов, мы будем вычислять методом linalg.norm() тоже из бибилиотеки NumPy.

In [109]:
def similar_user(new_user_stats, users_stats):
    
    # вычисляем евклидову норму методом .linalg.norm() для нового пользователя
    new_user_length = np.linalg.norm(new_user_stats)
    cosine_similarity = []

    for user in users_stats:
        # выяисляем евклидову норму методом .linalg.norm() для пользователя из базы данных
        old_user_length = np.linalg.norm(user)
        
        # вычисляем скалярное произведение методом .dot() и делим на умноженную евклидову норму нового пользователя и пользователя из базы данных
        cosinus = np.dot(new_user_stats, user) / (new_user_length*old_user_length)

        # косинусное сходство добавляем в список
        cosine_similarity.append(cosinus)
        
    # выводим индекс максимального косинуса из списка
    return f'Пользователь из базы с индексом: {cosine_similarity.index(max(cosine_similarity))} похож на нового пользователя'